### Instructions:
##### Part 1:
Write two data descriptors:
- IntegerField -> only allows `integral` numbers between `minimum` and `maximum` value.
- CharField -> only allows `strings` with a `minimum` and `maximum` length.

Assume we're not using slots.

##### Part 2:

Refactor your code and createa a `BaseValidator` class that will handle the common functionality.

Then change your `IntegerField` and `CharField` descriptors to inherit from BaseValidator.


If you haven't coded your descriptors that way already, make sure you can also omit one or both of the `miminum` and `maximum` values where it makes sense.

For example we may want to specify a string field that has no `maximum` limit, or we may want an integer field that has an upper bound, but no lower bound, or maybe no bounds at all.

Don't forget unit tests!

### Part 1:

In [7]:
from numbers import Integral

class IntegerField:
    def __init__(self, min_value=None, max_value=None):
        self.min_value = min_value
        self.max_value = max_value
        
    def __set_name__(self, owner_class, name):
        self.name = name
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return instance.__dict__.get(self.name, None)
    
    def __set__(self, instance, value):
        if not isinstance(value, Integral):
            raise TypeError(f'{self.name} must be an integral number.')
        if self.min_value is not None and self.min_value > value:
            raise ValueError(f'{self.name} cannot be below {self.min_value}.')
        if self.max_value is not None and self.max_value < value:
            raise ValueError(f'{self.name} cannot be greater than {self.max_value}.')
        instance.__dict__[self.name] = value
        
class CharField:
    def __init__(self, min_value=None, max_value=None):
        self.min_value = min_value
        self.max_value = max_value
        
    def __set_name__(self, owner_class, name):
        self.name = name
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return instance.__dict__.get(self.name, None)
    
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise TypeError(f'{self.name} must be a string.')
        if self.min_value is not None and self.min_value > len(value):
            raise ValueError(f'{self.name} cannot be shorter than {self.min_value} characters.')
        if self.max_value is not None and self.max_value < len(value):
            raise ValueError(f'{self.name} cannot be longer than {self.max_value} characters.')
        instance.__dict__[self.name] = value

In [8]:
class TestClass:
    number = IntegerField(min_value=2, max_value=100.1)
    string = CharField(min_value=1, max_value=25)
    
    def __init__(self, test_number, test_string):
        self.number = test_number
        self.string = test_string

In [21]:
t = TestClass(5, '1234567890123456789012345')
assert t.number == 5
assert t.string == '1234567890123456789012345'
t.number = 4
assert t.number == 4
t.string = '1'
assert t.string == '1'

In [24]:
try:
    t = TestClass(1, 'hello')
except ValueError as ex:
    assert 'cannot be below' in str(ex)

In [25]:
try:
    t = TestClass(200, 'hello')
except ValueError as ex:
    assert 'cannot be greater' in str(ex)

In [26]:
try:
    t = TestClass(2, '')
except ValueError as ex:
    assert 'cannot be shorter' in str(ex)

In [27]:
try:
    t = TestClass(2, '12345678901234567890123456')
except ValueError as ex:
    assert 'cannot be longer' in str(ex)

In [28]:
try:
    t = TestClass('2', '12345678901234567890123456')
except TypeError as ex:
    assert 'integral' in str(ex)

In [29]:
try:
    t = TestClass(2, 3)
except TypeError as ex:
    assert 'string' in str(ex)

### Part 2:

In [57]:
from numbers import Integral

class BaseValidator:
    def __init__(self, min_value=None,
                 max_value=None,
                 input_type=None,
                 validation_type='value'):
        self.min_value = min_value
        self.max_value = max_value
        self.input_type = input_type
        self.validation_type = validation_type
        
    def __set_name__(self, owner_class, name):
        self.name = name
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return instance.__dict__.get(self.name, None)
    
    def validate(self, value):
        pass
    
    def __set__(self, instance, value):
        self.validate(value)
        instance.__dict__[self.name] = value
        
class IntegerField(BaseValidator):
    def __init__(self, min_value=None, max_value=None):
        super().__init__(min_value=min_value,
                         max_value=max_value,
                         input_type=Integral)
        
    def validate(self, value):
        if not isinstance(value, self.input_type):
            raise TypeError(f'{self.name} must be of type {self.input_type.__name__}.')
        if self.min_value is not None and self.min_value > value:
            raise ValueError(f'Minimum allowable {self.validation_type} for {self.name} is {self.min_value}.')
        if self.max_value is not None and self.max_value < value:
            raise ValueError(f'Maximum allowable {self.validation_type} for {self.name} is {self.max_value}.')
        
class CharField(BaseValidator):
    def __init__(self, min_value=None, max_value=None):
        super().__init__(min_value=min_value,
                         max_value=max_value,
                         input_type=str,
                         validation_type='length')
        
    def validate(self, value):
        if not isinstance(value, self.input_type):
            raise TypeError(f'{self.name} must be of type {self.input_type.__name__}.')
        if self.min_value is not None and self.min_value > len(value):
            raise ValueError(f'Minimum allowable {self.validation_type} for {self.name} is {self.min_value}.')
        if self.max_value is not None and self.max_value < len(value):
            raise ValueError(f'Maximum allowable {self.validation_type} for {self.name} is {self.max_value}.')

In [58]:
class TestClass:
    number = IntegerField(min_value=2, max_value=100.1)
    string = CharField(min_value=1, max_value=25)
    
    def __init__(self, test_number, test_string):
        self.number = test_number
        self.string = test_string

In [59]:
t = TestClass(5, '1234567890123456789012345')
assert t.number == 5
assert t.string == '1234567890123456789012345'
t.number = 4
assert t.number == 4
t.string = '1'
assert t.string == '1'

In [60]:
try:
    t = TestClass(1, 'hello')
except ValueError as ex:
    assert 'Minimum allowable value' in str(ex)

In [61]:
try:
    t = TestClass(200, 'hello')
except ValueError as ex:
    assert 'Maximum allowable value' in str(ex)

In [62]:
try:
    t = TestClass(2, '')
except ValueError as ex:
    assert 'Minimum allowable length' in str(ex)

In [63]:
try:
    t = TestClass(2, '12345678901234567890123456')
except ValueError as ex:
    assert 'Maximum allowable length' in str(ex)

In [64]:
try:
    t = TestClass('2', '12345678901234567890123456')
except TypeError as ex:
    assert 'type Integral' in str(ex)

In [65]:
try:
    t = TestClass(2, 3)
except TypeError as ex:
    print(ex)
    assert 'type str' in str(ex)

string must be of type str.
